In [47]:
from bs4 import BeautifulSoup as bs
import re
from db import DB, Event, EventType, EventIs
import requests
from bs4 import BeautifulSoup
import json
import re

from db import DB, DB_INIT
DB_INIT(remove=True)

with open('data/events_94-2016.html', 'r') as infile:
    page = infile.read()

s = bs(page, 'html.parser')

def race_types(cells):
    racetypes = [t.strip() for t in cells[3].find_all(text=True)]
    race_cat = [t.strip() for t in cells[3].find_all(text=True) if 'Category - ' in t]
    race_type = [t.strip() for t in cells[3].find_all(text=True) if 'Category - ' not in t]
    return race_cat, race_type
    

getdata = dict()
getdata['event_name'] = (lambda cells: cells[1].find('b').get_text().strip()) # event Name
getdata['location'] = (lambda cells: cells[1].find(text = re.compile("^[a-zA-Z .]+, [A-Z]{2}")).strip())
getdata['dates'] = (lambda cells: cells[1].find(text = re.compile("\s+\d{2}/\d{2}/\d{4}")).strip())
getdata['flyer'] = (lambda cells: cells[1].find('a', href=True, text='Event Flyer')['href'])
getdata['event_website'] = (lambda cells: cells[1].find('a', href=True, text='Event Website')['href'])
getdata['permit_number'] = (lambda cells: cells[1].find(text=re.compile("\s+Permit Number:\s\S+")).strip().split(': ')[1])
getdata['online_reg'] = (lambda cells: cells[1].find('a', href=True, text='Online Registration')['href'])
getdata['promoter'] = (lambda cells: cells[2].find('a', href=True,).get_text().strip())
getdata['director'] = (lambda cells: cells[2].find('a', href='javascript:void(0)',).get_text().strip())


row = s.find('table').find('tr')  # Search result row
# print('*** {}'.format(row.get_text()))
row = row.find_next_sibling() # Column header row
# print('*** {}'.format(row.get_text()))
#These are the event rows.
for r in row.find_next_siblings():
    if 'Try another search' not in r.get_text():
        cells = r.find_all('td', recursive=False)
        rcat, rtype = race_types(cells)
        rowdata = dict()
        row['race_cat'] = rcat
        for key, l in getdata.items():
            try:
                rowdata[key] = l(cells)
            except:
                rowdata[key] = ''
        try:
            if rowdata['event_name']:
                ev = Event.create(**rowdata)
        except Exception as e:
            print(e)
        try:
            for t in rtype:
                et, created = EventType.get_or_create(raceType=t)
                EventIs.create(anEvent=ev, anEventType=et)
        except Exception as e:
            print(e)
            raise

In [19]:
def race_types(cells):
    """
    Parse race type column in events row
    :param cells:
    :return:
    """
    race_cat = [t.strip() for t in cells[3].find_all(text=True) if 'Category - ' in t]
    race_type = [t.strip() for t in cells[3].find_all(text=True) if 'Category - ' not in t]
    return race_cat, race_type

def parse_event_row(row):
    getdata = dict()
    getdata['event_name'] = (lambda cells: cells[1].find('b').get_text().strip()) # event Name
    getdata['location'] = (lambda cells: cells[1].find(text = re.compile("^[a-zA-Z .]+, [A-Z]{2}")).strip())
    getdata['dates'] = (lambda cells: cells[1].find(text = re.compile("\s+\d{2}/\d{2}/\d{4}")).strip())
    getdata['flyer'] = (lambda cells: cells[1].find('a', href=True, text='Event Flyer')['href'])
    getdata['event_website'] = (lambda cells: cells[1].find('a', href=True, text='Event Website')['href'])
    getdata['permit_number'] = (lambda cells: cells[1].find(text=re.compile("\s+Permit Number:\s\S+")).strip().split(': ')[1])
    getdata['online_reg'] = (lambda cells: cells[1].find('a', href=True, text='Online Registration')['href'])
    getdata['promoter'] = (lambda cells: cells[2].find('a', href=True,).get_text().strip())
    getdata['director'] = (lambda cells: cells[2].find('a', href='javascript:void(0)',).get_text().strip())

    cells = row.find_all('td', recursive=False)
    rcat, rtype = race_types(cells)
    rowdata = dict()
    row['race_cat'] = rcat
    for key, l in getdata.items():
        try:
            rowdata[key] = l(cells)
        except:
            rowdata[key] = ''
    return rowdata, rtype

def load_events(htmlfile):
    """
    This is for bulk loading of events. This is not designed for updating the database.
    The html file data/events_94-15.html has all events available 1994 through the end of 2015
    :param htmlfile:
    :return:
    """
    with open(htmlfile, 'r') as infile:
        page = infile.read()
    s = bs(page, 'html.parser')

    row = s.find('table').find('tr')  # Search result row
    # print('*** {}'.format(row.get_text()))
    row = row.find_next_sibling() # Column header row
    # print('*** {}'.format(row.get_text()))
    count = 0
    for r in row.find_next_siblings(): #These are the event rows.
        if 'Try another search' not in r.get_text():
            try:
                rowdata, rtype = parse_event_row(r)
            except Exception as e:
                print(count)
                raise
            try:
                if rowdata['event_name']:
                    ev = Event.create(**rowdata)
            except Exception as e:
                print(e)
            try:
                for t in rtype:
                    et, created = EventType.get_or_create(raceType=t)
                    EventIs.create(anEvent=ev, anEventType=et)
            except Exception as e:
                print(count)
                print(e)
                raise
        count += 1

from usactool.db import DB, DB_INIT
DB_INIT(remove=True)                
load_events('usactool/data/events_page_example.html')

No events for state AK and year 2013
No events for state ND and year 2013
No events for state WY and year 2013
No events for state AK and year 2014
No events for state ND and year 2014
No events for state WY and year 2014

In [5]:
# scrape past events pages
%load_ext autoreload
%autoreload 2
import logging
import importlib
importlib.reload(logging)

from usactool.tools import *
from usactool.db import DB, DB_INIT
DB_INIT(remove=True)  
get_past_events(2005, 2016, states, pageloc='URL', fileloc='usactool/data/past/', delay=3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
No events for state AK and year 2010
race_cat error:
[<td class="homearticlebody" valign="top"><img height="22" src="/images/smallrank1.gif" width="20"/></td>, <td class="homearticlebody" valign="top"><b>Tour de Olmstead</b>  <br>
				Permit Number: 2010-1854  <br>
				06/12/2010<br>Iroquois Park<br>Louisville,, KY<br><a href="getflyer.php?permit=2010-1854" target="_blank">Event Flyer</a><br><a href="http://www.olmstedparks.org/tourdeolmsted.html" target="_blank">Event Website</a><br><table bgcolor="#FFFFCC" border="0" width="100%"><tr>
<td class="homearticlebody"><b><a href="/results/?permit=2010-1854">View Results</a></b></td></tr></table><br/></br></br></br></br></br></br></br></td>, <td class="homearticlebody" valign="top"><a href="/clubs/index.php?club=9472">2WheelSports</a><br><a href="javascript:void(0)" onclick='getRDRating(46200,"Michael Hewitt")'>Michael Hewitt</a>  <div class="current_rat

In [16]:
import logging
logger = logging.getLogger()
logger.filename='example.log'
logger.setLevel(logging.DEBUG)
logger.debug("test")
logger.debug('This message should go to the log file')
logger.info('So should this')
logger.warning('And this, too')


DEBUG:root:test
DEBUG:root:This message should go to the log file
INFO:root:So should this


In [25]:
import logging
import importlib
importlib.reload(logging)
logging.basicConfig(filename='example.log',level=logging.DEBUG)
logging.debug('This message should go to the log file')
logging.info('So should this')
logging.warning('And this, too')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
No events for state AK and year 2005
No events for state ND and year 2005
No events for state SD and year 2005
No events for state AK and year 2006
No events for state ND and year 2006
No events for state SD and year 2006
No events for state WV and year 2006
No events for state AK and year 2007
No events for state DC and year 2007
No events for state ND and year 2007
No events for state AK and year 2008
No events for state ND and year 2008
No events for state AK and year 2009
race_cat error:
[<td class="homearticlebody" valign="top"><img height="22" src="/images/smallrank1.gif" width="20"/></td>, <td class="homearticlebody" valign="top"><b>Tour Des Lacs</b>  <br>
				Permit Number: 2009-2115  <br>
				08/08/2009<br>1006 N. Leroy St<br>Fenton, MI<br><a href="getflyer.php?permit=2009-2115" target="_blank">Event Flyer</a><br><a href="http://www.tourdeslacs.org" target="_blank">Event Website</a><br><br/></br></br></br></br></br></br></br></td>, <td class="homearticlebody" valign="top"><a href="/clubs/index.php?club=11311">Cyclefit Sports Club</a><br><a href="javascript:void(0)" onclick='getRDRating(177087,"Ron Stack")'>Ron Stack</a>  <div class="current_rating" onclick="getRDRating(177087,'Ron Stack')"><img src="/images/star_unselected.png" width="15"/><img src="/images/star_unselected.png" width="15"/><img src="/images/star_unselected.png" width="15"/><img src="/images/star_unselected.png" width="15"/><img src="/images/star_unselected.png" width="15"/></div><br><script language="javascript" type="text/javascript">
var e = unescape("%3Ec%22jtgh%3F%24ocknvq%3CMko%22Hqgtuvgt%22%3EmhqgtuvgtBejctvgt0pgv%40%24%40Mko%22Hqgtuvgt%22%3EmhqgtuvgtBejctvgt0pgv%40%3E1c%40");
var i,p='';for(i=0;i<e.length;i++){p+=String.fromCharCode(((e.charCodeAt(i)-34)%240)+32);}
document.write(p);
</script><noscript>Kim Foerster <kfoerster <em="">-a<!-- 133361841 -->t-</kfoerster></noscript></br></br></td>]
race_type error:
[<td class="homearticlebody" valign="top"><img height="22" src="/images/smallrank1.gif" width="20"/></td>, <td class="homearticlebody" valign="top"><b>Tour Des Lacs</b>  <br>
				Permit Number: 2009-2115  <br>
				08/08/2009<br>1006 N. Leroy St<br>Fenton, MI<br><a href="getflyer.php?permit=2009-2115" target="_blank">Event Flyer</a><br><a href="http://www.tourdeslacs.org" target="_blank">Event Website</a><br><br/></br></br></br></br></br></br></br></td>, <td class="homearticlebody" valign="top"><a href="/clubs/index.php?club=11311">Cyclefit Sports Club</a><br><a href="javascript:void(0)" onclick='getRDRating(177087,"Ron Stack")'>Ron Stack</a>  <div class="current_rating" onclick="getRDRating(177087,'Ron Stack')"><img src="/images/star_unselected.png" width="15"/><img src="/images/star_unselected.png" width="15"/><img src="/images/star_unselected.png" width="15"/><img src="/images/star_unselected.png" width="15"/><img src="/images/star_unselected.png" width="15"/></div><br><script language="javascript" type="text/javascript">
var e = unescape("%3Ec%22jtgh%3F%24ocknvq%3CMko%22Hqgtuvgt%22%3EmhqgtuvgtBejctvgt0pgv%40%24%40Mko%22Hqgtuvgt%22%3EmhqgtuvgtBejctvgt0pgv%40%3E1c%40");
var i,p='';for(i=0;i<e.length;i++){p+=String.fromCharCode(((e.charCodeAt(i)-34)%240)+32);}
document.write(p);
</script><noscript>Kim Foerster <kfoerster <em="">-a<!-- 133361841 -->t-</kfoerster></noscript></br></br></td>]
No events for state ND and year 2009
No events for state AK and year 2010
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-6-8280b02b7df4> in <module>()
      6 from usactool.db import DB, DB_INIT
      7 DB_INIT(remove=True)
----> 8 get_past_events(2005, 2013, states, pageloc='URL', fileloc='usactool/data/past/', delay=3)

/Users/vincentdavis/VersionControl/USAC_data/usactool/tools.py in get_past_events(start, end, states, pageloc, fileloc, delay)
    126                 eventspage = req.get("http://www.usacycling.org/events/?state=" + state + "&race=&fyear=" + str(year) + "&rrfilter=rr" , headers=HDRS).text
    127                 if '<small><sub>(200)</sub></small>' in eventspage: # we are not getting all the results on this page.
--> 128                     raise
    129                 if fileloc:
    130                     with open('{}events_{}_{}'.format(fileloc, state, year), 'w') as f:

RuntimeError: No active exception to reraise